In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from dataset import SimulationData
import math
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\Pietro\anaconda3\envs\rans\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = SimulationData(stencilNum=500,samplePerStencil=100, fileNames=["aw=1.1_internal.csv","aw=1.2_internal.csv","aw=1.3_internal.csv", "aw=1.4_internal.csv","aw=1.5_internal.csv","aw=1.6_internal.csv","aw=1.7_internal.csv","aw=1.8_internal.csv","aw=1.9_internal.csv","aw=2.0_internal.csv"], override=True)
#dataset.normalizeData()

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [25:58<00:00,  3.12s/it]


In [3]:
class MLP(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            
            super(MLP, self).__init__()
            self.input_size = input_size
            self.hidden_size1  = hidden_size1
            self.hidden_size2 = hidden_size2
            self.output_size = output_size
            self.linear1 = nn.Linear(self.input_size, self.hidden_size1)
            #self.batchNorm1 = nn.BatchNorm2d(self.hidden_size1)
            self.relu1 = nn.ReLU()
            self.linear2 = nn.Linear(self.hidden_size1, self.hidden_size2)
            #self.batchNorm2 = nn.BatchNorm2d(self.hidden_size2)
            self.relu2 = nn.ReLU()
            self.linear3 = nn.Linear(self.hidden_size2, self.output_size)
            #self.batchNorm2 = nn.BatchNorm2d(self.hidden_size2)
            #self.relu3 = nn.ReLU()
            
        def forward(self, x):
            hidden1 = self.linear1(x)
            relu1  =self.relu1(hidden1)
            hidden2 = self.linear2(relu1)
            relu2 = self.relu2(hidden2)
            output = self.linear3(relu2)
            return output

In [4]:
class WeightedSum(nn.Module):
    def __init__(self):
        super(WeightedSum, self).__init__()
        self.G_split_size = 5
    
    def forward(self, G, input_features):
        #print(f'if: {input_features[0]}')
        G_ = G[:, :self.G_split_size]
        #print(G_.shape[0])
        L = (torch.matmul(torch.transpose(G, 1,0), input_features) / G.shape[0])
        L_ = (torch.matmul(torch.transpose(input_features, 1, 0), G_) / G.shape[0])
        D = torch.matmul(L, L_)
        #print(f'D: {D}')
        #print()
        return D
    

In [5]:
class MainNet(nn.Module):
    def __init__(self, input_size1, hidden_size1_1,hidden_size1_2, output_size1,
                 input_size2, hidden_size2_1, hidden_size2_2, output_size2):
        
        super(MainNet, self).__init__()
        self.invariant_features_indeces = [4,5,6,7]

        self.embeddingNetwork = MLP(input_size1,hidden_size1_1, hidden_size1_2, output_size1)
        self.weightedSum = WeightedSum()
        self.flatten = nn.Flatten(1,-1)
        self.fittingNetwork = MLP(input_size2,hidden_size2_1, hidden_size2_2, output_size2)
        
        
   
    def forward(self, x):
    
        #print(f'input values: {x[0]}\n shape: {x.shape}')
        invariant_x = x[:, self.invariant_features_indeces]
        
        embedding_output = self.embeddingNetwork(invariant_x)
        #print(f' shape: {embedding_output.shape}')

        weighted_output = self.weightedSum(embedding_output, x)
        #print(f' shape: {weighted_output.shape}')
        
        weighted_output_flattened = torch.flatten(weighted_output)
        #print(f' shape: {weighted_output_flattened.shape}')
        
        fitting_output = self.fittingNetwork(weighted_output_flattened)
        #print(f'fitting output: {fitting_output}\n shape: {fitting_output.shape}')

        return fitting_output

In [6]:
input_size1 = 4
hidden_size1_1 = 32
hidden_size1_2 = 64
output_size1 = 256

input_size2 = 256*5
hidden_size2_1 = 64
hidden_size2_2 = 32
output_size2 = 3

net = MainNet(input_size1, hidden_size1_1, hidden_size1_2, output_size1, input_size2,hidden_size2_1, hidden_size2_2, output_size2)

In [7]:
len(dataset)

4946

In [8]:
import math
test_perc = 0.22
train_perc = 1-test_perc
train_length = math.ceil(len(dataset) * train_perc)
test_length = math.ceil(len(dataset) *test_perc)
train_length = train_length + (len(dataset)-train_length-test_length)
train, test = torch.utils.data.random_split(dataset, [train_length,test_length], generator=torch.Generator().manual_seed(42))

In [9]:
loader_train = DataLoader(dataset= train, batch_size = 1, num_workers=1)
loader_test = DataLoader(dataset= test, batch_size = 1, num_workers=1)

In [10]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

criterion = nn.MSELoss()

In [11]:
"""
def test():
    net.eval()
    with torch.no_grad():
        losses = []
        counter = 0
        for i, data in enumerate(loader_test):
            counter +=1
            x_batch,y_batch = data
            for i in range(len(x_batch)):
                x = x_batch[i]
                y = y_batch[i]
                output1, output2, output3 = net(x.float())
                loss1 = criterion(output1.reshape(1).float(), y[[0]].float())
                loss2 = criterion(output2.reshape(1).float(), y[[1]].float())
                loss3 = criterion(output3.reshape(1).float(), y[[2]].float())
                loss = ((loss1)+(loss2)+(loss3)/3)
                losses.append(loss.item())
    print(f'avg testing loss: {sum(losses)/len(losses)}')
    return losses
"""

"\ndef test():\n    net.eval()\n    with torch.no_grad():\n        losses = []\n        counter = 0\n        for i, data in enumerate(loader_test):\n            counter +=1\n            x_batch,y_batch = data\n            for i in range(len(x_batch)):\n                x = x_batch[i]\n                y = y_batch[i]\n                output1, output2, output3 = net(x.float())\n                loss1 = criterion(output1.reshape(1).float(), y[[0]].float())\n                loss2 = criterion(output2.reshape(1).float(), y[[1]].float())\n                loss3 = criterion(output3.reshape(1).float(), y[[2]].float())\n                loss = ((loss1)+(loss2)+(loss3)/3)\n                losses.append(loss.item())\n    print(f'avg testing loss: {sum(losses)/len(losses)}')\n    return losses\n"

In [12]:
from dataset import rotate

def test_rotation(x, model):
    with torch.no_grad():
        #print()
        #print(f'input: {x[0]}')

        output1, output2, output3 = model(x.float())
        rotate_cloud(x)
        output1_rotated, output2_rotated, output3_rotated = model(x.float())

    #print(f'input rotated: {x[0]}')
    '''
    print(f'{output1} - {output1_rotated} # pass: {(output1 - output1_rotated) <=0.0001}')
    print(f'{output2} - {output2_rotated} # pass: {(output2 - output2_rotated) <=0.0001}')
    print(f'{output3} - {output3_rotated} # pass: {(output3 - output3_rotated) <=0.0001}')
    '''
def rotate_cloud(x):
    for point in x:
        point[[0,1]] = rotate(-90,point[[0,1]])
        point[[2,3]] = rotate(-90,point[[2,3]])
        point[[8,9]] = rotate(-90,point[[8,9]])
    return x



In [13]:
def train_one_epoch():
    net.train()
    losses = []
    losses_k = []
    losses_eps = []
    losses_nut = []
    
    losses_eval = []
    losses_k_eval = []
    losses_eps_eval = []
    losses_nut_eval = []
    info = []
    counter = 0

    for i, data in enumerate(zip(loader_train, loader_test)):
        counter +=1
        loader_train_temp,loader_test_temp = data
        x_batch, y_batch = loader_train_temp
        x_batch_test, y_batch_test = loader_test_temp
        info.append({})
        for i in range(len(x_batch)):
            #print(x_batch)
            x = x_batch[i].float()
            y = y_batch[i].float()
            #print(x_batch.grad)
            #print(y)
            x_test = x_batch_test[i].float()
            y_test = y_batch_test[i].float()
            
            #test_rotation(x, net)
            #test_rotation(x_test, net)
            
            info[counter-1]["x"] = x
            info[counter-1]["y"] = y
            
            optimizer.zero_grad()
            output1, output2, output3 = net(x)
            loss1 = criterion(output1.reshape(1), y[[0]])
            loss2 = criterion(output2.reshape(1), y[[1]])
            loss3 = criterion(output3.reshape(1), y[[2]])
            loss = ((loss1)+(loss2)+(loss3))/3
            
            '''
            print('-')
            print(f'output:{output1.reshape(1).float()} - true: { y[[0]].float()}')
            print(f'output:{output2.reshape(1).float()} - true: { y[[1]].float()}')
            print(f'output:{output3.reshape(1).float()} - true: { y[[2]].float()}')
            print('-')
            print(loss)
            '''
            
            info[counter-1]["output"] = [output1.item(),output2.item(),output3.item()]
            info[counter-1]["loss"] = loss.item()
            info[counter-1]["loss epsilon"] = loss1.item()
            info[counter-1]["loss k"] = loss2.item()
            info[counter-1]["loss nut"] = loss3.item()
            #print(net.fittingNetwork.linear3.weight)

            loss.backward()
            optimizer.step()
            #print(net.fittingNetwork.linear3.weight)

            with torch.no_grad():
                output1, output2, output3 = net(x_test.float())
                loss1_eval = criterion(output1.reshape(1).float(), y_test[[0]].float())
                loss2_eval = criterion(output2.reshape(1).float(), y_test[[1]].float())
                loss3_eval = criterion(output3.reshape(1).float(), y_test[[2]].float())
                loss_eval = ((loss1_eval)+(loss2_eval)+(loss3_eval))/3

            #print(x[0])
            #print(output1.item(), output2.item(), output3.item())
            #print(y[0].item(), y[1].item(), y[2].item())
            #print(loss.item())
            #print()
            #if(loss.item()>=0.20):
            losses.append(loss.item())
            losses_eps.append(loss1.item())
            losses_k.append(loss2.item())
            losses_nut.append(loss3.item())
            
            losses_eval.append(loss_eval.item())
            losses_eps_eval.append(loss1_eval.item())
            losses_k_eval.append(loss2_eval.item())
            losses_nut_eval.append(loss3_eval.item())

    return losses,losses_k,losses_eps, losses_nut,losses_eval,losses_eps_eval,losses_k_eval,losses_nut_eval, info

In [14]:
train_losses = []
train_losses_k = []
train_losses_eps = []
train_losses_nut = []

test_losses = []
test_losses_k = []
test_losses_eps = []
test_losses_nut= []

infos_train = []
infos_val = []

In [ ]:
for i in tqdm(range(200)):
    loss,loss_k, loss_eps, loss_nut,loss_eval,loss_eps_eval,loss_k_eval,loss_nut_eval, info = train_one_epoch()
    
    train_losses.extend(loss)
    train_losses_nut.extend(loss_nut)
    train_losses_eps.extend(loss_eps)
    train_losses_k.extend(loss_k)
    
    test_losses.extend(loss_eval)
    test_losses_nut.extend(loss_nut_eval)
    test_losses_eps.extend(loss_eps_eval)
    test_losses_k.extend(loss_k_eval)
    
    infos_train.extend(info)
    
    print('#############')
    print(f'epoch {i}:')
    print(f'total training loss {sum(loss) / len(loss)} total eval loss {sum(loss_eval) / len(loss_eval)}')
    print(f'nut training loss {sum(loss_nut) / len(loss_nut)} nut eval loss {sum(loss_nut_eval) / len(loss_nut_eval)}')
    print(f'eps training loss {sum(loss_eps) / len(loss_eps)} eps eval loss {sum(loss_eps_eval) / len(loss_eps_eval)}')
    print(f'k training loss {sum(loss_k) / len(loss_k)} k eval loss {sum(loss_k_eval) / len(loss_k_eval)}')
    print('#############')
    print()

    

  0%|▍                                                                                 | 1/200 [00:11<38:46, 11.69s/it]

#############
epoch 0:
total training loss 0.11901866704607124 total eval loss 0.18285657235412547
nut training loss 0.14515400652783828 nut eval loss 0.21282743057784353
eps training loss 0.09857042520117924 eps eval loss 0.15051489845899316
k training loss 0.11333156879281461 k eval loss 0.18522738510541198
#############




  1%|▊                                                                                 | 2/200 [00:25<43:14, 13.10s/it]

#############
epoch 1:
total training loss 0.1247795376262083 total eval loss 0.1852594947844788
nut training loss 0.15224610330548527 nut eval loss 0.21681452627767642
eps training loss 0.10364852195974644 eps eval loss 0.1464150067174314
k training loss 0.1184439871580947 k eval loss 0.1925489539366074
#############




  2%|█▏                                                                                | 3/200 [00:42<48:59, 14.92s/it]

#############
epoch 2:
total training loss 0.13459334468199338 total eval loss 0.19937319821727598
nut training loss 0.15744560933953802 nut eval loss 0.2277894773414009
eps training loss 0.10341451908927785 eps eval loss 0.14636505950057882
k training loss 0.1429199039373273 k eval loss 0.22396505723161647
#############



In [ ]:
def averageOut(list_nums, n):
    new_list = []
    temp_sum = 0
    temp_length = 0
    counter = 0
    average = 0
    temp_list = []
    final_list = []
    for i, num in enumerate(list_nums):
        counter+=1
        temp_list.append(num)
        if counter == n:
            avg = sum(temp_list) / len(temp_list)
            final_list.append(avg)
            temp_list = []
            counter = 0
    if len(temp_list) != 0:
        avg = sum(temp_list) / len(temp_list)
        final_list.append(avg)
    return final_list

In [ ]:
plt.yscale("log")
smoothing_factor = 1089

plt.plot(averageOut(train_losses, smoothing_factor), color="red", label="training loss")
plt.plot(averageOut(test_losses, smoothing_factor), color="blue", label="validation loss")

#ax2.plot(averageOut(train_losses_k, smoothing_factor), color="red", label="training loss")
#ax2.plot(averageOut(test_losses_k, smoothing_factor), color="blue", label="validation loss")
#ax3.plot(averageOut(train_losses_nut, smoothing_factor), color="red", label="training loss")
#ax3.plot(averageOut(test_losses_nut, smoothing_factor), color="blue", label="validation loss")
plt.title("validation & training loss (log scale)")
plt.xlabel("iteration")
plt.ylabel("loss")
plt.legend()
plt.savefig("./images/loss_net_C_4.0=500_S=100_E=300.png", dpi=1500)

In [ ]:
len(train_losses)/1089.0

In [ ]:
torch.save(net.state_dict(), "./models/net_C_4.0=500_S=100_E=300.pth")


In [ ]:
net.load_state_dict(torch.load("./models/net.pth"))